# LookUp Climate Data from EU xr.DataSets for Austrian Locations

### Presettings

In [11]:
import os
import re
import datetime as dt
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

from scipy import stats

In [2]:
root       = "C:/Users/freiste/OneDrive - Ilmatieteen laitos/Documents/IIASA YSSP 2023"

input_dir  = f"{root}/02 - Data/EU/BOKU_ImprovedForestCharacteristics"
output_dir = f"{root}/02 - Data/AUT"


subfolders = [
    "Age",
    "Alive_Tree_Carbon",
    "Basal_Area",
    "Biomass",
    "Diameter",
    "Height",
    "Stand_Density_Index",
    "Stem_Number",
    "Tree_Species_Group",
    "Volume"
    ]




### Methods

In [8]:
# get all certain-type files inside a specific folder

def show_all_files(input_dir, typ='.csv'):

    regexp = re.compile(fr"{typ}")
    files  = [path.name for path in os.scandir(input_dir) if path.is_file() if regexp.search(path.name)]
    
    return files


# Postprocessing

### Create Skeleton

In [4]:
# Begin from Geographical Details

FS = pd.read_excel(f'{output_dir}/Data_Geography.xlsx')

droplist = ['Area', 'Population','PopulationDensity', 'NumberOfMunicipalities',
            'lat_center1','lon_center1', 'elev_center1', 'lat_center2', 'lon_center2',
            'elev_center2', 'elev_center']
FS.drop(columns=droplist, index=[0, 85], inplace=True)

In [6]:
# AUT Climate Table Skeleton
skeleton_order = ['ForestryDistrict', 'FederalState', 'lat_center', 'lon_center']
FS = FS[skeleton_order]
FS

,ForestryDistrict,FederalState,lat_center,lon_center
1,Eisenstadt+Rust,Burgenland,47.82915,16.67885
2,Güssing,Burgenland,47.113836,16.320419
3,Jennersdorf,Burgenland,46.98195,16.197311
4,Mattersburg,Burgenland,47.728369,16.418644
5,Neusiedl/See,Burgenland,47.900129,16.951993
...,...,...,...,...
80,Bregenz,Vorarlberg,47.411097,9.937755
81,Dornbirn,Vorarlberg,47.416538,9.733014
82,Feldkirch,Vorarlberg,47.257412,9.664555
83,Vorarlberg,Vorarlberg_total,47.25,9.916667


## Look up data by lat & lon and write it into Skeleton

In [28]:
def lookup_coor_val(i):

    val =\
    FSDS.sel(longitude = FS.lon_center[i],
             latitude  = FS.lat_center[i],
             method    = 'nearest').Band1.values

    return val

In [29]:
for sub in subfolders:
    
    folder = f'{input_dir}/{sub}'
    
    files = show_all_files(folder, typ='_EPSG4326.nc')  # getting the right projection
    
    for file in files:
        print(file)
        
        FSDS = xr.open_dataset(f"{folder}/{file}")
        FSDS.close()
        
        # Add column of this variable to Skeleton
        # Lookup coordinates in the BOKU files
        varname = re.findall('(.+)_s?d?_?EPSG4326.nc', file)
        FS[varname] = FS.index.map(lookup_coor_val)
        
        
        
        
        

agecl_1_perc_EPSG4326.nc


KeyError: "'longitude' is not a valid dimension or coordinate"

In [30]:
FS

,ForestryDistrict,FederalState,lat_center,lon_center
1,Eisenstadt+Rust,Burgenland,47.82915,16.67885
2,Güssing,Burgenland,47.113836,16.320419
3,Jennersdorf,Burgenland,46.98195,16.197311
4,Mattersburg,Burgenland,47.728369,16.418644
5,Neusiedl/See,Burgenland,47.900129,16.951993
...,...,...,...,...
80,Bregenz,Vorarlberg,47.411097,9.937755
81,Dornbirn,Vorarlberg,47.416538,9.733014
82,Feldkirch,Vorarlberg,47.257412,9.664555
83,Vorarlberg,Vorarlberg_total,47.25,9.916667


In [ ]:
i = 2
print(Geo.lon_center[i])
print(Geo.lat_center[i])
FSDS.sel(longitude= Geo.lon_center[i],
         latitude = Geo.lat_center[i],
         method='nearest').Band1#.values

In [26]:
s = 'agecl_3_perc_sd_EPSG4326.nc'
varname = re.findall('(.+)_s?d?_?EPSG4326.nc', s)
varname

['agecl_3_perc_sd']